# Lab | Hypothesis Testing

**Objective**

Welcome to the Hypothesis Testing Lab, where we embark on an enlightening journey through the realm of statistical decision-making! In this laboratory, we delve into various scenarios, applying the powerful tools of hypothesis testing to scrutinize and interpret data.

From testing the mean of a single sample (One Sample T-Test), to investigating differences between independent groups (Two Sample T-Test), and exploring relationships within dependent samples (Paired Sample T-Test), our exploration knows no bounds. Furthermore, we'll venture into the realm of Analysis of Variance (ANOVA), unraveling the complexities of comparing means across multiple groups.

So, grab your statistical tools, prepare your hypotheses, and let's embark on this fascinating journey of exploration and discovery in the world of hypothesis testing!

**Challenge 1**

In this challenge, we will be working with pokemon data. The data can be found here:

- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/pokemon.csv

In [42]:
#libraries
import pandas as pd
import scipy.stats as st
import numpy as np



In [43]:
df = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/pokemon.csv")
df

,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False
1,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False
2,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False
3,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,False
4,Charmander,Fire,NaN,39,52,43,60,50,65,1,False
...,...,...,...,...,...,...,...,...,...,...,...
795,Diancie,Rock,Fairy,50,100,150,100,150,50,6,True
796,Mega Diancie,Rock,Fairy,50,160,110,160,110,110,6,True
797,Hoopa Confined,Psychic,Ghost,80,110,60,150,130,70,6,True
798,Hoopa Unbound,Psychic,Dark,80,160,60,170,130,80,6,True


- We posit that Pokemons of type Dragon have, on average, more HP stats than Grass. Choose the propper test and, with 5% significance, comment your findings.

In [44]:
df[(df['Type 1'] == 'Grass') & (df['Type 2'] == 'Dragon')]

,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
275,Mega Sceptile,Grass,Dragon,70,110,75,145,85,145,3,False


In [45]:
#code here

# H0 ; hp[dragon] mu <= hp[grass] mu
# H1 L hp[dragon] mu > hp[grass] mu
alpha = 0.05
dragon_hp = df[(df['Type 1'] == 'Dragon') | (df['Type 2'] == 'Dragon')]['HP']
grass_hp = df[(df['Type 1'] == 'Grass') | (df['Type 2'] == 'Grass')]['HP']

stat, p_value = st.ttest_ind(dragon_hp,grass_hp,alternative='greater', equal_var=False)
print(f"{stat = }, {p_value = }")
if p_value < alpha and stat > 0:
    print('Reject Null Hypothesis: Dragon average HP > Grass average HP')
else:
    print("Fail to reject Null Hypothesis: Dragon average HP <= Grass average HP")

stat = 4.097528915272702, p_value = 5.090769061176926e-05
Reject Null Hypothesis: Dragon average HP > Grass average HP


using 2 sample test one tailed greater, I find that dragons do have more hp on average than grass types.

- We posit that Legendary Pokemons have different stats (HP, Attack, Defense, Sp.Atk, Sp.Def, Speed) when comparing with Non-Legendary. Choose the propper test and, with 5% significance, comment your findings.


In [46]:
#code here
alpha = 0.05
# H0 ; stats legendary mu == stats non_lengendary mu
# H1 L stats legendary mu != stats non_legendary
legendary_stats = df[df['Legendary'] == True][['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']]
non_legendary_stats = df[df['Legendary'] == False][['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']]

legendary_base_stats = legendary_stats.sum(axis=1)
non_legendary_base_stats = non_legendary_stats.sum(axis=1)

stat, p_value = st.ttest_ind(legendary_base_stats,non_legendary_base_stats, equal_var=False)
print(f"{stat = }, {p_value = }")
if p_value < alpha:
    print('Reject Null Hypothesis: Legendary Pokémon have different stats from Non-Legendary Pokémon')
else:
    print('Fail to reject Null Hypothesis: There is not enough evidence to conclude that Legendary Pokémon have different stats from Non-Legendary Pokémon')

stat = 25.8335743895517, p_value = 9.357954335957446e-47
Reject Null Hypothesis: Legendary Pokémon have different stats from Non-Legendary Pokémon


Assuming we should sum our base stats among pokemon, and using 2 sample test two=sided, I find that legendary do have different stats than non-legendary

**Challenge 2**

In this challenge, we will be working with california-housing data. The data can be found here:
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/california_housing.csv

In [47]:
df = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/california_housing.csv")
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-114.31,34.19,15.0,5612.0,1283.0,1015.0,472.0,1.4936,66900.0
1,-114.47,34.40,19.0,7650.0,1901.0,1129.0,463.0,1.8200,80100.0
2,-114.56,33.69,17.0,720.0,174.0,333.0,117.0,1.6509,85700.0
3,-114.57,33.64,14.0,1501.0,337.0,515.0,226.0,3.1917,73400.0
4,-114.57,33.57,20.0,1454.0,326.0,624.0,262.0,1.9250,65500.0


**We posit that houses close to either a school or a hospital are more expensive.**

- School coordinates (-118, 37)
- Hospital coordinates (-122, 34)

We consider a house (neighborhood) to be close to a school or hospital if the distance is lower than 0.50.

Hint:
- Write a function to calculate euclidean distance from each house (neighborhood) to the school and to the hospital.
- Divide your dataset into houses close and far from either a hospital or school.
- Choose the propper test and, with 5% significance, comment your findings.


In [48]:
import math
def euclid_dist(point1, point2):
    return math.sqrt(sum([(x - y) ** 2 for x, y in zip(point1, point2)]))

In [55]:
def is_close_to_home(row):
    threshold = .50
    house_coords = (row['longitude'], row['latitude'])
    school_coords = (-118, 37)
    hospital_coords = (-122, 34)

    dist_school = euclid_dist(house_coords, school_coords)
    dist_hospital = euclid_dist(house_coords, hospital_coords)
    return dist_school < threshold or dist_hospital < threshold

df['close_to_home'] = df.apply(is_close_to_home, axis=1)

house_val = 'median_house_value'
far_from_home = df[df['close_to_home'] == False]
close_to_home = df[df['close_to_home'] == True]
alpha = .05

t_stat, p_value = st.ttest_ind(close_to_home[house_val], far_from_home[house_val], alternative='greater', equal_var=False)
print(f"{t_stat = }, {p_value = }")
if p_value < alpha and t_stat > 0:
    print('Reject Null Hypothesis: houses close to either a school or a hospital are more expensive.')
else:
    print('Fail to reject Null Hypothesis.')

t_stat = -17.174167998688404, p_value = 0.9999738999071939
Fail to reject Null Hypothesis.


Using two sample test two-sided, and through calculating the euclidean distance between houses and the school/hospital, I don't find evidence to support that the houses closer to school/hospital are more expensive than those that aren't